# 0. Import Packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

29596


# 1. Load Data

## Load the spot df folder
The initially picked spots are generated from the notebook: [2_jie_pick_spot](2_jie_pick_spot.ipynb)



In [2]:
para_subfolder =r'CW_v2'

main_folder =r'\\10.245.74.158\Chromatin_NAS_8\Exported_data\jie_CW_decode'
picked_spot_folders = [os.path.join(main_folder, _fl) for _fl in os.listdir(main_folder) if 'exp' in _fl and '_clean' not in _fl]
picked_spot_folders = [os.path.join(_fl,para_subfolder) for _fl in picked_spot_folders]
picked_spot_folders

['\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220316exp\\CW_v2',
 '\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220402exp\\CW_v2',
 '\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220419exp\\CW_v2',
 '\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220713exp\\CW_v2']

# 2. Import functions

The modified jie is the same version as described below except that the packages are named differently from the ones in the repository:

The link to get [jie_CW](https://github.com/cosmosyw/jie/tree/0f797efcc52c1e9822a3cd03d8980cba9315b468) 

The link to get [jie_CW.aligner_SL](https://github.com/cosmosyw/jie/blob/0f797efcc52c1e9822a3cd03d8980cba9315b468/jie/aligner.py)

In [3]:
import jie_CW
from jie_CW.jie import aligner_SL
from jie_CW.jie import plot

Add function **sort_spots_by_fiber_density** below if it is not in the [jie_CW.aligner_SL](https://github.com/cosmosyw/jie/blob/0f797efcc52c1e9822a3cd03d8980cba9315b468/jie/aligner.py)

In [ ]:
# Function to sort picked spots based on (local) fiber density, this would help resolve less probable trans-homolog transversing (like "fiber swapping during tracing")
def sort_spots_by_fiber_density (picked_chr_pts_cell, _iter_num_th = 30, _local_reg_num = 30):
    #import scipy
    from scipy.spatial.distance import cdist
    from itertools import permutations

    picked_chr_pts_cell_new =  picked_chr_pts_cell.copy(deep=True)
    chosen_chrom = picked_chr_pts_cell_new['chr'].values[0]
    cell_id = picked_chr_pts_cell_new['orig_cellID'].values[0]

    if 'fiberidx_jie' not in picked_chr_pts_cell_new.columns:
        picked_chr_pts_cell_new['fiberidx_jie']=picked_chr_pts_cell_new['fiberidx']

    # process if more than one fiber exists for a cell
    if len(np.unique(picked_chr_pts_cell_new['fiberidx'])) >1:
        sel_pts_zxyhfa = picked_chr_pts_cell[['z_hat','x_hat','y_hat','hyb','fiberidx']].copy(deep=True)
        _iter = 0
        while _iter <_iter_num_th:
            _iter +=1
            curr_fiberidx = sel_pts_zxyhfa['fiberidx'] # flag the current fiberidx
            picked_hyb_list = sorted(np.unique(sel_pts_zxyhfa['hyb']))
            # 10% of the longest picked fiber as local ref 
            _neighbor_len = round(np.max(picked_hyb_list)/5)
            for _hyb in picked_hyb_list:
                # sel_pts_zxyhfa below will be updated after processing each hyb
                _cand_spots = sel_pts_zxyhfa[sel_pts_zxyhfa['hyb']==_hyb]
                # local neighboring region
                _start, _end = max(_hyb - _neighbor_len, np.min(picked_hyb_list)), min(_hyb + _neighbor_len, np.max(picked_hyb_list)) 
                ref_ct_list = []
                # assign spots to closest fiber
                for _fiber_id in np.unique(sel_pts_zxyhfa['fiberidx']): # order of fibers to use
                    sel_pts_cluster = sel_pts_zxyhfa[sel_pts_zxyhfa['fiberidx']==_fiber_id]
                    _local_hyb_list = np.intersect1d(np.arange(_start, _end+1), np.unique(sel_pts_cluster['hyb']))
                    # use local center if there are enough nearby spots
                    if len(_local_hyb_list) >= _local_reg_num:
                        ref_spots = sel_pts_cluster[sel_pts_cluster['hyb'].isin(_local_hyb_list)]
                    # use all the decoded chr fiber center
                    else:
                        ref_spots = sel_pts_cluster.copy(deep=True)
                    ref_ct = np.nanmean(ref_spots[['z_hat','x_hat','y_hat']],axis=0) # local center coords
                    ref_ct_list.append(ref_ct)
                # dist matrix for cand_spots and ref_centers
                ref_cts = np.array(ref_ct_list)
                _dist_mtx =  cdist(_cand_spots[['z_hat','x_hat','y_hat']],ref_cts)
                # re-assign for fibers based on the min summed distance
                _cand_spots_new = _cand_spots.copy(deep=True)
                _l = list(permutations(range(0, len(ref_cts)),len(_dist_mtx)))
                _sum_dist_list = []
                for _i in _l:
                    # distance sum for this permutation
                    _dist_sum = 0
                    for _spot_index in range(len(_dist_mtx)):
                        _dist_sum += _dist_mtx [_spot_index,_i[_spot_index]] 
                    _sum_dist_list.append(_dist_sum)
                ########################################## 
                # pick the permutation with lowest summed distance
                _exchanged_index = _l[np.argmin(_sum_dist_list)]
                _cand_spots_new['fiberidx']= np.unique(sel_pts_zxyhfa['fiberidx'])[list(_exchanged_index)]
                # assign back to the sel_pts_zxyhfa for the next hyb
                sel_pts_zxyhfa_new = sel_pts_zxyhfa.copy(deep=True)
                sel_pts_zxyhfa_new[sel_pts_zxyhfa_new['hyb']==_hyb] = _cand_spots_new
                sel_pts_zxyhfa = sel_pts_zxyhfa_new.copy(deep=True)

            # check reshuffling rate
            num_change = np.sum(sel_pts_zxyhfa['fiberidx']!=curr_fiberidx)
            rate_change = num_change/len(sel_pts_zxyhfa)
            if rate_change<=0.001:
                break
                
        print(f'Complete sorting for {chosen_chrom} in {cell_id} within {_iter} iteration(s).')
        picked_chr_pts_cell_new[['z_hat','x_hat','y_hat','hyb','fiberidx']]=sel_pts_zxyhfa[['z_hat','x_hat','y_hat','hyb','fiberidx']]
        
    # skip if only one fiber
    else:
        print(f'Skip sorting for {chosen_chrom} in {cell_id} because there is only one fiber.')
        
    return picked_chr_pts_cell_new
            

In [4]:
import importlib
importlib.reload(aligner_SL)
help(aligner_SL.sort_spots_by_fiber_density)

Help on function sort_spots_by_fiber_density in module jie_CW.jie.aligner_SL:

sort_spots_by_fiber_density(picked_chr_pts_cell, _iter_num_th=30, _local_reg_num=30)
    # Function to sort picked spots based on (local) fiber density, this would help resolve less probable trans-homolog transversing (like "fiber swapping during tracing")



In [5]:
import multiprocessing as mp
import tqdm

num_threads =20

# 3. Sort the picked spots for each exp replicate

## Exp 0713

### load data

In [6]:
dna_exp_key= '20220713'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220713exp\\CW_v2'

In [7]:
output_df_dict.keys()

dict_keys(['20220713exp_pick_chr1.h5', '20220713exp_pick_chr10.h5', '20220713exp_pick_chr11.h5', '20220713exp_pick_chr12.h5', '20220713exp_pick_chr13.h5', '20220713exp_pick_chr14.h5', '20220713exp_pick_chr15.h5', '20220713exp_pick_chr16.h5', '20220713exp_pick_chr17.h5', '20220713exp_pick_chr18.h5', '20220713exp_pick_chr19.h5', '20220713exp_pick_chr2.h5', '20220713exp_pick_chr3.h5', '20220713exp_pick_chr4.h5', '20220713exp_pick_chr5.h5', '20220713exp_pick_chr6.h5', '20220713exp_pick_chr7.h5', '20220713exp_pick_chr8.h5', '20220713exp_pick_chr9.h5', '20220713exp_pick_chrX.h5'])

In [8]:
output_df_dict[list(output_df_dict.keys())[0]]

,replicate,FOV,orig_cellID,finalcellID,celltype,chr,hyb,region_name,x_hat,y_hat,z_hat,sig_x,sig_y,sig_z,rowID,fiberidx
23,20220713,18,Cell-10,100502394973884731031710640222794322905,Astro,chr1,12,1:21732182-21745770,259.372640,620.405783,39.411673,1,1,1,23521942,1
27,20220713,18,Cell-10,100502394973884731031710640222794322905,Astro,chr1,14,1:23749258-23759965,262.756717,623.565353,44.769108,1,1,1,23520207,1
34,20220713,18,Cell-10,100502394973884731031710640222794322905,Astro,chr1,16,1:24241577-24251162,261.750523,626.064167,42.448645,1,1,1,23522032,1
44,20220713,18,Cell-10,100502394973884731031710640222794322905,Astro,chr1,21,1:35544821-35559274,257.991170,622.192240,45.894748,1,1,1,23522005,1
51,20220713,18,Cell-10,100502394973884731031710640222794322905,Astro,chr1,25,1:38751230-38759983,254.111773,627.143457,41.999315,1,1,1,23518889,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,20220713,39,Cell-17,9522942335541362634092199696394126659,Vip,chr1,145,1:188751447-188759965,582.260823,101.583803,66.213708,1,1,1,35691805,1
470,20220713,39,Cell-17,9522942335541362634092199696394126659,Vip,chr1,148,1:189510684-189521211,573.796500,101.345085,59.822325,1,1,1,35694674,1
477,20220713,39,Cell-17,9522942335541362634092199696394126659,Vip,chr1,150,1:191250765-191259922,571.806520,103.273113,60.417039,1,1,1,35691122,1
478,20220713,39,Cell-17,9522942335541362634092199696394126659,Vip,chr1,151,1:193105714-193114935,579.318605,98.332320,61.322402,1,1,1,35695299,1


### process data

In [9]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner_SL.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


6695it [00:00, 6831.63it/s]


Start multiprocessing chromosome chr1
Finish processing chromosome chr1
00:04:40.50
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7237it [00:01, 6331.36it/s]


Start multiprocessing chromosome chr10
Finish processing chromosome chr10
00:02:55.64
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8
Finish processing chromosome chr8
00:02:20.64
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6371it [00:00, 6409.46it/s] 


Start multiprocessing chromosome chr9
Finish processing chromosome chr9
00:01:59.19
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


4178it [00:00, 9900.48it/s] 


Start multiprocessing chromosome chrX
Finish processing chromosome chrX
00:00:06.20
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Exp 0316

### load data

In [38]:
dna_exp_key= '20220316'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220316exp\\CW_v2'

In [39]:
output_df_dict.keys()

dict_keys(['20220316exp_pick_chr1.h5', '20220316exp_pick_chr10.h5', '20220316exp_pick_chr11.h5', '20220316exp_pick_chr12.h5', '20220316exp_pick_chr13.h5', '20220316exp_pick_chr14.h5', '20220316exp_pick_chr15.h5', '20220316exp_pick_chr16.h5', '20220316exp_pick_chr17.h5', '20220316exp_pick_chr18.h5', '20220316exp_pick_chr19.h5', '20220316exp_pick_chr2.h5', '20220316exp_pick_chr3.h5', '20220316exp_pick_chr4.h5', '20220316exp_pick_chr5.h5', '20220316exp_pick_chr6.h5', '20220316exp_pick_chr7.h5', '20220316exp_pick_chr8.h5', '20220316exp_pick_chr9.h5', '20220316exp_pick_chrX.h5'])

### process data

In [40]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner_SL.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


6143it [00:01, 5610.05it/s]


Start multiprocessing chromosome chr1
Finish processing chromosome chr1
00:03:41.77
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6920it [00:00, 7828.06it/s] 


Start multiprocessing chromosome chr10
Finish processing chromosome chr10
00:02:57.56
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8
Finish processing chromosome chr8
00:01:55.83
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


5886it [00:00, 8384.62it/s] 


Start multiprocessing chromosome chr9
Finish processing chromosome chr9
00:01:38.06
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


2890it [00:00, 4459.88it/s]


Start multiprocessing chromosome chrX
Finish processing chromosome chrX
00:00:04.90
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Exp 0402

### load data

In [41]:
dna_exp_key= '20220402'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220402exp\\CW_v2'

In [42]:
output_df_dict.keys()

dict_keys(['20220402exp_pick_chr1.h5', '20220402exp_pick_chr10.h5', '20220402exp_pick_chr11.h5', '20220402exp_pick_chr12.h5', '20220402exp_pick_chr13.h5', '20220402exp_pick_chr14.h5', '20220402exp_pick_chr15.h5', '20220402exp_pick_chr16.h5', '20220402exp_pick_chr17.h5', '20220402exp_pick_chr18.h5', '20220402exp_pick_chr19.h5', '20220402exp_pick_chr2.h5', '20220402exp_pick_chr3.h5', '20220402exp_pick_chr4.h5', '20220402exp_pick_chr5.h5', '20220402exp_pick_chr6.h5', '20220402exp_pick_chr7.h5', '20220402exp_pick_chr8.h5', '20220402exp_pick_chr9.h5', '20220402exp_pick_chrX.h5'])

### process data

In [43]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner_SL.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


5810it [00:01, 5481.13it/s]


Start multiprocessing chromosome chr1
Finish processing chromosome chr1
00:04:46.12
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6316it [00:00, 7674.36it/s] 


Start multiprocessing chromosome chr10
Finish processing chromosome chr10
00:03:01.71
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8
Finish processing chromosome chr8
00:01:59.28
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


5481it [00:00, 8254.52it/s]


Start multiprocessing chromosome chr9
Finish processing chromosome chr9
00:02:00.52
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


4010it [00:00, 9709.44it/s] 


Start multiprocessing chromosome chrX
Finish processing chromosome chrX
00:00:06.21
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


## Exp 0419

### load data

In [44]:
dna_exp_key= '20220419'

# output dict for filename, file
output_df_dict = {}

exp_picked_spot_folder = [_f for _f in picked_spot_folders if dna_exp_key in _f][0]

if os.path.exists(exp_picked_spot_folder):
    file_list = [os.path.join(exp_picked_spot_folder, _fl) for _fl in os.listdir(exp_picked_spot_folder) if '.h5' in _fl]
    for file in file_list:
        output_df_dict[os.path.basename(file)]=pd.read_hdf(file)
        
exp_picked_spot_folder

'\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\jie_CW_decode\\20220419exp\\CW_v2'

In [45]:
output_df_dict.keys()

dict_keys(['20220419exp_pick_chr1.h5', '20220419exp_pick_chr10.h5', '20220419exp_pick_chr11.h5', '20220419exp_pick_chr12.h5', '20220419exp_pick_chr13.h5', '20220419exp_pick_chr14.h5', '20220419exp_pick_chr15.h5', '20220419exp_pick_chr16.h5', '20220419exp_pick_chr17.h5', '20220419exp_pick_chr18.h5', '20220419exp_pick_chr19.h5', '20220419exp_pick_chr2.h5', '20220419exp_pick_chr3.h5', '20220419exp_pick_chr4.h5', '20220419exp_pick_chr5.h5', '20220419exp_pick_chr6.h5', '20220419exp_pick_chr7.h5', '20220419exp_pick_chr8.h5', '20220419exp_pick_chr9.h5', '20220419exp_pick_chrX.h5'])

### process data

In [46]:
# define output folder
output_folder = exp_picked_spot_folder.replace(para_subfolder, f'{para_subfolder}_sorted')
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print ('Generating output folders')
    
# loop through all chr h5-df    
for fname, chr_df_all in output_df_dict.items():
    
    output_fname = os.path.join(output_folder, fname)
    chosen_chrom = chr_df_all['chr'].values[0]
    
    chr_df_cell_list = []
    # group by celltype, finalcellD
    for i, (name, group) in tqdm.tqdm(enumerate(chr_df_all.groupby(by=['celltype','finalcellID']))):
        chr_df_cell = copy.deepcopy(group)
        chr_df_cell_list.append(chr_df_cell)
        
    del chr_df_all
    print(f'Start multiprocessing chromosome {chosen_chrom}')
    start_time = time.time()
    with mp.Pool(num_threads) as sort_pool:
        _res = sort_pool.starmap(aligner_SL.sort_spots_by_fiber_density, zip(chr_df_cell_list), chunksize=1)
        sort_pool.close()
        sort_pool.join()
        sort_pool.terminate()

    elapsed_time = time.time() - start_time
    print(f'Finish processing chromosome {chosen_chrom}')
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    sorted_chr_df_all = pd.concat(_res)
    sorted_chr_df_all.to_hdf(output_fname, key='df', mode='w')
    #del(sorted_chr_df_all)
    print(f'Finish saving chromosome {chosen_chrom}.')
    print(f'+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    


Generating output folders


6708it [00:01, 5863.63it/s]


Start multiprocessing chromosome chr1
Finish processing chromosome chr1
00:03:32.80
Finish saving chromosome chr1.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


7443it [00:01, 5846.82it/s]


Start multiprocessing chromosome chr10
Finish processing chromosome chr10
00:02:27.11
Finish saving chromosome chr10.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


...

Start multiprocessing chromosome chr8
Finish processing chromosome chr8
00:01:41.38
Finish saving chromosome chr8.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


6407it [00:00, 8519.95it/s] 


Start multiprocessing chromosome chr9
Finish processing chromosome chr9
00:01:24.33
Finish saving chromosome chr9.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


3203it [00:00, 9795.12it/s] 


Start multiprocessing chromosome chrX
Finish processing chromosome chrX
00:00:05.31
Finish saving chromosome chrX.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
